# Creating More Complex Programs

-----

This goal of this lesson is to introduce the user to some more advanced features using Pyquil beyond simply creating and measuring pre-defined programs.

Before proceeding, please consider reading the intro lesson in this series, which covers all of the basics of programs and measurements in Pyquil:

Lesson 1 - Intro to Programs and Measurements

https://github.com/dkoch92/Quantum-Algorithm-Tutorials

---

If you wish to skip around this lesson for the examples of code, please run the following cell of code below to ensure all functions are imported properly:

In [120]:
import numpy as np
from pyquil.quil import Program
from pyquil.gates import I, H
from pyquil.api import QVMConnection

def SD(qubit_index,total_qubits):
    standard = int( abs( qubit_index - (total_qubits-1) ) )
    return standard

qvm = QVMConnection()

## Flexability in Programs
---

In the previous lesson, we have already seen how to create basic programs, which are essentialy our quantum algorithm's 'list of instructions'.  The example below shows the extent to which we have covered programs to far:

In [294]:
perfect_coins = Program( H(0),H(1) ).measure(0,0).measure(1,1)
results = qvm.run(perfect_coins,[0,1],1)[0]

print('coins: ',results)

coins:  [0, 1]


To recap, the flow of the code above is as follows:
    
1) We create a quantum state of 2 qubit, where each qubit is initialized with a Hadamard gate:   Program( H(0),H(1) )

2) We then add a measurement to our program of qubit 0, storing the measurement result in the classical register index 0: .measure(0,0)

3) We then do the same thing for qubit 1: .measure(1,1)

4) We then run the program 1 time, and check the classical register indices 0 and 1 afterwards: qvm.run(perfect_coins,[0,1],1)

5) Lastly, we extract the results from the classical register in the form of an array: qvm.run(...)[0]

For any clarification on these steps, please refer to lesson 1 in this series for more detailed explinations

---
### Amending Programs
---

In the example above, the program 'perfect_coins' is fully prepared in one line of code.  We can check this by simply printing it to console:

In [22]:
perfect_coins = Program( H(0),H(1) ).measure(0,0).measure(1,1)
print(perfect_coins)

H 0
H 1
MEASURE 0 [0]
MEASURE 1 [1]



#### .inst()
---

While there are certinaly many instances where we want to prepare and measure a quantum system as a static program, Pyquil provides us with some tools for creating more dynamic algorithms.  The first of which is a function called .inst(), from the quil.py file.  This function allows us to add any additional instructions to our program, wherever we like.

We have already seen a similar thing with the .measure() function from the previous lesson, but .inst() allows us to ammend any instruction we want to the program, even measurements!:

In [304]:
three_qubits = Program( I(0) )

print('___Initialized___')
print(three_qubits)
print(qvm.wavefunction(three_qubits))
print('  ')

three_qubits.inst( H(1),I(2) )
print('___After inst( H(1),I(2) )__')
print(three_qubits)
print(qvm.wavefunction(three_qubits))
print(' ')

three_qubits.inst("MEASURE 1 [1]")
print('___After inst("MEASURE 1 [1]")___')
print(three_qubits)
print(qvm.wavefunction(three_qubits))

___Initialized___
I 0

(1+0j)|0>
  
___After inst( H(1),I(2) )__
I 0
H 1
I 2

(0.7071067812+0j)|000> + (0.7071067812+0j)|010>
 
___After inst("MEASURE 1 [1]")___
I 0
H 1
I 2
MEASURE 1 [1]

(1+0j)|010>


In the example above, we create an initial program that only contains 1 qubit.  Then, we use the .inst() function to add on 2 more qubits, and later a measurement to our program.  The wavefunctions printed below each program confirm that the .inst() function is working as intended.

.inst() has quite a bit of flexability in taking arguments as well.  Consider all of the following ways we can append two qubits:

In [36]:
three_qubits = Program( I(0) )
three_qubits.inst( I(1) ,I(2) )
print(' Multiple Instructions: ',qvm.wavefunction(three_qubits))

three_qubits = Program( I(0) )
three_qubits.inst( [I(1) ,I(2)] )
print(' Array of Instructions: ',qvm.wavefunction(three_qubits))

three_qubits = Program( I(0) )
three_qubits.inst( I(i) for i in range(1,3) )
print('              for loop: ',qvm.wavefunction(three_qubits))

three_qubits = Program( I(0) )
three_qubits.inst( [("I",1),("I",2)] )
print('       Array of Tuples: ',qvm.wavefunction(three_qubits))

three_qubits = Program( I(0) )
three_qubits.inst( ["I 1","I 2"] )
print('      Array of Strings: ',qvm.wavefunction(three_qubits))

three_qubits = Program( I(0) )
qubits_to_add = Program( I(1),I(2) )
three_qubits.inst( qubits_to_add)
print('Adding Another Program: ',qvm.wavefunction(three_qubits))

 Multiple Instructions:  (1+0j)|000>
 Array of Instructions:  (1+0j)|000>
              for loop:  (1+0j)|000>
       Array of Tuples:  (1+0j)|000>
      Array of Strings:  (1+0j)|000>
Adding Another Program:  (1+0j)|000>


Holy moly that's  alot.  The .inst() function gives us a great deal of flexability in the style we want to append instructions to a program.  But at the end of the day, whether we feed the function strings, tuples, arrays, etc., many of these methods are repetative, only differing in syntax.

However, I will point out that the two examples from the code above that are the most noteworthy are $\textit{for loop}$ and $\textit{Adding Another Program}$.  The other four examples are all sort of 'different flavors' of accomplishing the same task.

The ability to have a for-loop as a set of instructions is quite handy if we want to insert a long list of instructions, in a nice compact line of code.  Similarly, the ability to ammend an entire second program is a great way of adding lots of instructions at once.  It also gives us the freedom to work with multiple programs throughout the code, choosing if and when we want to ammend 'secondary programs' to our main program.  Consider the following code, which utilizes both methods to create a nice compact code of 6 qubits:

In [41]:
six_qubits = Program()

qubits_0_thru_2 = Program(I(i) for i in range(3))
qubits_3_thru_4 = Program(H(3),H(4))
six_qubits.inst([qubits_0_thru_2, qubits_3_thru_4, I(5)] )

print(six_qubits)
print(qvm.wavefunction(six_qubits))

I 0
I 1
I 2
H 3
H 4
I 5

(0.5+0j)|000000> + (0.5+0j)|001000> + (0.5+0j)|010000> + (0.5+0j)|011000>


Okay, the example above is a bit over contrived, but it drives the point home.  The final program 'six_qubits' is created using a variety of the tools available to us via the .inst() function.  Also note that I used the for-loop as an argument to the Program() function, rather than the inst() function.  As it turns out, all of the argument examples provided above $\textit{also}$ work as arguments to the Program() function.

If we consider that the only job of the .inst() function is to append things to a program, it would make sense that .inst() and Program() take all of the same arguments.  I won't bore you with the same long example to prove this (you're welcome), but I encourage you to try all of the arguments shown above in Program() for yourself.

## Aside - Pyquil's State Notation
---

Before continuing, I would like to take a moment here to address a notation choice by Pyquil.  You may have noticed that the state created in the example isn't what you may have been expecting.  This is because Pyquil's .wavefunction() displays states' different from standard notation.  For reference, the 'standard' convention for the ordering of qubits is typically from left to right:

state |$1101\rangle \rightarrow$ qubit 0 = |$1\rangle$  ,  qubit 1 = |$1\rangle$  ,  qubit 2 = |$0\rangle$   ,  qubit 3 = |$1\rangle$ 

However, Pyquil's notation reads from right to left:

state |$1100\rangle \rightarrow$ qubit 0 = |$0\rangle$  ,  qubit 1 = |$0\rangle$  ,  qubit 2 = |$1\rangle$   ,  qubit 3 = |$1\rangle$ 

The following code below shows an example of this:

In [316]:
qubit_notation = Program( H(0),I(1),I(2) )

print('qubit 0:  H      qubit 1:  I      qubit 2:   I')
print('___wavefunction___')
print(qvm.wavefunction(qubit_notation))

qubit 0:  H      qubit 1:  I      qubit 2:   I
___wavefunction___
(0.7071067812+0j)|000> + (0.7071067812+0j)|001>


Running the code above, and noting which qubit recieved the Hadamard gate, you should notice the notation issue.  Well, 'issue' is sort of a strong word - there's actually nothing wrong with the way Pyquil chooses to label their qubits.  At the end of the day, .wavefunction() is just a tool for helping to visualize wavefunctions.  When we tell a program to apply an operation is qubit 0, Pyquil does exactly that. 

Nevertheless, for learning purposes it may be helpful to some if the .wavefunction() matched standard notation, so let's add a simple work around!  The defined function below is a simple one line of code that remedies the notation confusion:

In [318]:
def SD(qubit_index,total_qubits):
    standard = int( abs( qubit_index - (total_qubits-1) ) )
    return standard

q0 = SD(0,3)
q1 = SD(1,3)
q2 = SD(2,3)
qubit_notation = Program( H(q0),I(q1),I(q2) )
results = qvm.run(qubit_notation,[0,1,2],1)[0]

print('qubit 0:  H      qubit 1:  I      qubit 2:   I')
print('___wavefunction___')
print(qvm.wavefunction(qubit_notation))
print('  ')

print('__program instructions__')
print(qubit_notation)

qubit 0:  H      qubit 1:  I      qubit 2:   I
___wavefunction___
(0.7071067812+0j)|000> + (0.7071067812+0j)|100>
  
__program instructions__
H 2
I 1
I 0



As we can see, now .wavefunction() matches our notation for the qubits.  However, we can see that now the program instructions are backwards!  Well, this is how we 'tricked' the wavefunction - we just reversed the numbering index on all the qubits.  So even though in our code we can think of qubit 0 as 'q0', Pyquil is actually performing operations on qubit 2.  This is because q0 = SD(0,2) $\rightarrow$ q0 = 2.

Unfortunately, we can't have our cake and eat it too with this quick fix -- either the .wavefunction() states will be backwards or our program instructions will.  We $\textit{could}$ work a little harder (but who does that?) and come up with a more comprehensive function that handles both instances - so we always print things in standard convention... but we won't.  Maybe in a future lesson! 

Please note that the function provided above is a completely cosmetic addition to our algorithms, so when we print the wavefunction, what we see if what we expect.  In a real quantum algorithm, there would be no need for such a function, but we will use it throughout these lessons for teaching purposes.

## More Ammending
---

Alrighty, back to business.  We've seen that .inst() allows us to add on instructions to our program, and now we will look at a function that does the opposite.

#### .pop()
----

.pop() is a function that removes the last instruction in a prpgram.  It 'pops it off' according to Pyquil's notation (I don't know why i love this function's name so much).  Let's see an example:

In [58]:
pop_off = Program(I(0),H(1),H(2))

print('Before pop: ')
print(pop_off)
pop_off.pop()
print(' After pop: ')
print(pop_off)

Before pop: 
I 0
H 1
H 2

 After pop: 
I 0
H 1



Simple enough.  .pop() is a function that doesn't take any arguments, so it will always remove the last instrution of a program.  

Now suppose that we don't want to remove the last instruction, but some intermediate one.  Since there is no built in Pyquil function (to my knowledge) that will allow us to do this, we need to create our own:

In [100]:
import numpy as np

def pop_x(prog,index):
    '''
    Uses the pop function to remove an instruction at a given index location
    '''
    length = len(prog)
    if(index==0):
        pf = Program(prog[1:length])
    elif(index==(length-1)):
        pf = Program(prog[0:(length-1)])
    else:
        p1 = Program(prog[0:int(index)])
        p2 = Program(prog[int(index+1):length])
        pf = p1 + p2
    return pf

too_many = Program( I(0),H(1),I(2),I(3) ).measure(0,0)
print(too_many)

just_right = pop_x( too_many,1 )
print(just_right)


I 0
H 1
I 2
I 3
MEASURE 0 [0]

I 0
I 2
I 3
MEASURE 0 [0]



The function pop_x above does the trick!  We start off with a program that initializes 4 qubits, tell pop_x() to remove the instruction with index 1, and sure enough the instrustion H(1) from the program.  Now let's go back and see how we did this, as we need to examine some more properties of programs to fully understand what is going on.

First off, one property of programs is that we can check them much like arrays:

In [99]:
program = Program(H(0),I(1),H(2))

print(program)
print(program[1])
print(program[0:2])

H 0
I 1
H 2

I 1
[<Gate H 0>, <Gate I 1>]


The examples above show that we can call on the values within a program using brackets [], just like a list or array.  So long as we use proper syntax, we can look at any instructions or section of instructions we want.

Pyquil comes with a nice pre-built functioanlity when we print a program as a whole (as we've seen several times), as shown by the first example.  This nice printing feature also works if we want to look at a single instruction, shown in the second example.  However, if we call upon a section of the program larger than 1, we get back an array of elements, third example.

Printing aside, whether we call upon one element or a list of elements, the objects are all the same: gates and measurements.  Thus, we can directly pull out these objects if we want, and use them for other purposes.  The code below does this by calling upon a subset of a program's instructions, and then assigning them to a $\textit{new}$ program:

In [322]:
program1 = Program(H(0),I(1),H(2))
print('___original program___')
print(program1)

program2 = Program( program1[0:2] )
print('___program2___')
print(program2)
print(qvm.wavefunction(program2))

___original program___
H 0
I 1
H 2

___program2___
H 0
I 1

(0.7071067812+0j)|00> + (0.7071067812+0j)|01>


Important to note here that in order to create a new program, like 'program2' above, we must pass the elements we borrowed from 'program1' into the function Program().

---

program2 = Program( program1[0:2] )

---

Consider the example below, which shows that instruction elements alone do not make a program:

In [324]:
program1 = Program(H(0),I(1),H(2))

program2 = program[0:2]
program3 = Program(program[0:2])

print('___program2___')
print(program2)
print('program2 type: ',type(program2))
print('  ')

print('___program3___')
print(program3)
print('program3 type: ',type(program3))
print('  ')

___program2___
[<Gate H 0>, <pyquil.quilbase.Measurement object at 0x000002439D03C390>]
program2 type:  <class 'list'>
  
___program3___
H 0
MEASURE 0

program3 type:  <class 'pyquil.quil.Program'>
  


We can see that program2 is a list of instructions, but it is not a program.  If we were to try and use either qvm.wavefunction() or qvm.run() on program2, we would get an error.

Now, using this new knowledge of how to create new programs from existing ones, we can return to the function pop_x() (was that what we were doing?).  The function pop_x essentially creates a new program, excluding the index we choose to remove:

---
p1 = Program(prog[0:int(index)])

p2 = Program(prog[int(index+1):length])
        
pf = p1 + p2
        
---

In fact, we don't even have to use the function .pop() at all!  We simply create two new programs, p1 and p2, which store all of the elements before and after the chosen index location, and then merge them back together: p1 + p2.

'p1 + p2'? You never told us we could do that!  Programs actually come with two nice built in features using '+' and '+='.  The first combines two programs together into one, while the second adds a program into an existing one:

In [291]:
p1 = Program(I(0),H(1))
p2 = Program(H(0),I(1))

print('____p1+p2____')
print(p1+p2 )

p1 += p2
print('____p1+=p2____')
print(p1)

p1 = Program(I(0),H(1))
p1.inst(p2)
print('____p1.inst(p2)____')
print(p1)

____p1+p2____
I 0
H 1
H 0
I 1

____p1+=p2____
I 0
H 1
H 0
I 1

____p1.inst(p2)____
I 0
H 1
H 0
I 1



The '+' operation is useful in combing two existing programs into a new program, while retaining the originals.  The '+=' operation ammends all of the instructions from a second program, onto the first.  This changes the first program, but leaves the second unchanged.  The '+=' function is essentially equivilant to the .inst() function, as shown above.

We are now done finished with out pop_x() exercise, but the real point was to showcase some of the deeper properties of programs (ha, I tricked you).  In more complex quantum algorithms, the ability to edit programs is essential, especially if we want to change what we do to the system based on measurement results, which we shall see next!

## Condition Branching
---

Up to this point, all of our programs have been very static  --  we create a program and then make a measurment.  We now have some nice tools for ammending our programs, but the whole process always concludes with a measurement.

Now suppose we want to $\textit{use}$ measurement results as an ingridient for future quantum algorithms.  For example, in the previous lesson we created a perfect coin function and used the results to determine the winner of a simple betting game.  This was an example of using quantum measurement results for a classical algorithm (the algorithm being count heads and tails to decide a winner).

What we would like to do now, is use quantum results to dictate quantum algorithms.  The example below shoes what I mean by this, using some standard python if-then statements:

In [333]:
q=[1,0]

part_two = Program(I(q[0]),I(q[1]))
coin = Program(H(q[0])).measure(q[0],0)
result = qvm.run(coin,[0],1)[0][0]

if(result==0):
    coin.inst(H(q[0]),I(q[1]))
else:
    coin.inst(I(q[0]),H(q[1]))

print('coin result: ',result)
print(qvm.wavefunction(coin))

coin result:  0
(0.7071067812+0j)|00> + (0.7071067812+0j)|10>


The code in this example uses the result from a quantum coin to determine what gates are applied to the program afterwards.  Specifically, the state that we find the coin qubit in, either |$0\rangle$ or |$1\rangle$, determines which qubit recieves a Hadamard gate in the following 2-qubit system.  Although kit is a simple example, this is what we mean by 'quantum results dictating quantum algorithms'.  Depending on the result of the first quantum system, our second quantum system is in one of two completely different states:

$$ \frac{1}{\sqrt{2}} ( |00\rangle + |10\rangle ) \hspace{.3cm} \textrm{or} \hspace{.3cm} \frac{1}{\sqrt{2}} ( |10\rangle + |11\rangle ) $$

Because these two systems have no overlapping states, a measurement on the second quantum system gives us full information about the first measurement! (*single clap*?)

Alright, let's take a look at how we can recreate the algorithm above using Pyquil's built in functions.  Rather than explicity going into the classical register for results and writing our own if-then statements, Pyquil does all this for us with the function: $\textbf{if_then}()$

if_then() takes three arguments $\rightarrow$ 1) a classical register index  2) a program to run if $\textit{true}$  3) a program to run if $\textit{false}$.

True and false for this function are determined by the value at the given classical register index.  1 = true, 0 = false.

In [368]:
q=[1,0]

coin2 = Program(H(q[0])).measure(q[0],q[0])
true_program  = Program(I(q[0]),H(q[1]))
false_program = Program(H(q[0]),I(q[1]))

coin2.if_then(q[0],true_program,false_program)
print(qvm.wavefunction(coin2))

(0.7071067812+0j)|10> + (0.7071067812+0j)|11>


Running the code above will result in one of two possible final states shown above, achieving same desired result from the previous example.  The if_then() function works similar to measure(), where it doesn't actually run the program, but simply ammends it.  Thus, depending on the first measurement result, the if_then() function ammends either true_program or false_program to 'coin2'.   We can see this by printing 'coin2' after the if_then() function:

In [371]:
q0 = SD(0,2)
q1 = SD(1,2)

coin = Program(H(q[0])).measure(q0,q0)
print('___before___')
print(coin)
true_program  = Program(I(q0),H(q1))
false_program = Program(H(q0),I(q1))

print('___after___')
print(coin.if_then(q0,true_program,false_program))

___before___
H 1
MEASURE 1 [1]

___after___
H 1
MEASURE 1 [1]
JUMP-WHEN @THEN1 [1]
H 1
I 0
JUMP @END2
LABEL @THEN1
I 1
H 0
LABEL @END2



As we can see, the if_then() function has added a significant amount of instructions to our coin program.  Let's disect each component (do we have too... yes!):

---

H 1

MEASURE 1 [1]

---

This is the original part of our coin program.  We begin by initializing qubit 1 with a Hadamard gate, measure it, and store the result in the classical register index [1].  Remember, we are using the SD() function so that our final wavefunction matches what we want: Hadamard on qubit 0 $\rightarrow$ H(q0) = "H 1".

---

JUMP-WHEN @THEN1 [1]

H 1

I 0

JUMP @END2

LABEL @THEN1

I 1

H 0

LABEL @END2

---

This is the set of instructions resulting from the if_then() function.  

Beginning from the top: 'JUMP-WHEN @THEN1 [1]' is the instruction to perform the $\textbf{true}$ condition.  The phrase JUMP-WHEN @THEN1 is the code's way of saying, "if true, go follow the instructions labeled 'THEN1'".  And what determines if this condition is true or false is shown at the end of this line by [1], which means, "use classical register index 1 to determine if true or false".

One of two things will happen next, following this JUMP-WHEN @ THEN1 line $\rightarrow$ 1) the classical register will be a $\textbf{1}$, which signals $\textbf{true}$, which means the code $\textit{skips}$ all of the code until reaching the line: 'LABEL @THEN1'.  2) the classical register will be $\textbf{0}$, which signals $\textbf{false}$, which means that the code proceeds to the next line down: 'H 1'.

A visual way of representing the lines of code above is as follows:

![title](If_Then.png)

Here, the short arrows represent jumps, which show the next line of code that is executed following the specified jump.

So long as we understand what the if_then() function is accomplishing: true $\rightarrow$ program1  |  false $\rightarrow$ program2, then we need not worry too much about the instructions elements.  The main point here is that the two conditional programs are being $\textit{added}$ to the original program, based a true/false condition.

The advantage to using the if_then() function is to make our algorithms easier, since it communicates with the classical register for us, rather than having to extract the measurement results ourselves.


## While_do
---

To go along with our new 'quantum' if-then function, Pyquil also comes with it's own pre-built while-loop: while_do().

While_do takes two arguments $\rightarrow$ 1) classical register index 2) program to run if $\textit{run}$.  Same as before, true is determined by the classical register index provided, where 1 = true.  The difference here is that this loop will keep repeating until the classical register index = 1.  Thus, like any while loop, we must be careful not to accidentally write anything that will go on infinitely!

Let's see an example:

In [452]:
coin = Program(H(0)).measure(0,0)
if_tails = Program(H(0)).measure(0,0)

coin.while_do(0,if_tails)
print(qvm.wavefunction(coin))

(1+0j)|0>


This example above always results in the qubit being in state |0$\rangle$.  For a better idea of what's going on, let's print the program to console:

In [453]:
coin = Program(H(0)).measure(0,0)
if_tails = Program(H(0)).measure(0,0)

coin.while_do(0,if_tails)
print(coin)

H 0
MEASURE 0 [0]
LABEL @START1
JUMP-UNLESS @END2 [0]
H 0
MEASURE 0 [0]
JUMP @START1
LABEL @END2



Just like the if_then() function, the while_do() function takes our program 'coin' and ammends onto it some new instructions.  Let's follow the logic of the instructions to see how the code jumps around:

---

LABEL @START1

JUMP-UNLESS @END2 [0]

H 0

MEASURE 0 [0]

JUMP @START1

LABEL @END2

---

LABEL @START1 $\rightarrow$ this is a marker that the code can jump to if called upon.

JUMP-UNLESS @END2 [0] $\rightarrow$ this is the same as our @JUMP-WHEN condition earlier.  This line of code can be understood as, "unless the classical register located at index [0] is equal to 1, jump to the location of the label @END2".  

This is where our program is branching based on a true/false condition: 1) If the classical register index value is $\textbf{0}$, which means $\textbf{false}$, then skip all code until we get to the designated label. 2) If the classical register index value is $\textbf{1}$, which means $\textbf{true}$, then proceed to the next line of code.

Now, for the case of $\textbf{true}$, the next three lines of code show why this process is potentially never ending:

---

H 0

MEASURE 0 [0]

JUMP @START1  $\hspace{.4cm}$    $\leftarrow$ here!

---

The first two lines of code are simply our 'if_tails' program.  But the third line of code is an instruction to jump, with no conditional on it.  Thus, when the code reaches this point, it will $\textit{always}$ jump to the line labeled @START1.  Looking up, that's the very beginning of the whole while_do() program!

The logic flow can be visualized as follows:

![title](While_do.png)

As illustrated by the excessively long looping arrow, for the case where the classical register indez is 1, the program will always loop back to the beginning of the while_do().  Thus, it is very important that there is something in that block of code that can change the classical register!  For this example, we pass the qubit through a Hadamard gate and measure again, which will $\textit{eventually}$ result in a measurement result of 0.

And that's all there is to it!  Just like the case of the if_then() function, the advantage of using while_do() is that it takes care of the communication with the classical register for us.

---

This concludes all of the topics for lesson 2!  I hope you enjoyed this lesson, and I encourage you to take a look at my other .ipynb tutorials!

link: https://github.com/dkoch92/Quantum-Algorithm-Tutorials